In [4]:
pip install scikit-learn

     ---------------------------------------- 9.2/9.2 MB 4.1 MB/s eta 0:00:00
     ---------------------------------------- 44.0/44.0 MB 2.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import pycrfsuite

from sklearn.preprocessing import LabelBinarizer
from itertools import chain
from sklearn.metrics import classification_report

def readCorpus(fpath):
    sents = []
    with open(fpath) as fd:
        sent = []
        for l in fd:
            #lt = l.strip().decode("utf8")
            lt = l.strip()
            if not lt:
                sents.append(sent)
                sent = []
            else:
                w_t = lt.split('\t')
                sent.append([w_t[0], w_t[1]])
    return sents

def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]
    features = [
        #'bias',
        'word.lower=' + word.lower(),
        #'word[-3:]=' + word[-3:],
        #'word[-2:]=' + word[-2:],
        #'word.isupper=%s' % word.isupper(),
        #'word.istitle=%s' % word.istitle(),
        #'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        #'postag[:2]=' + postag[:2],
    ]
    if i > 0:
        pass
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.extend([
            '-1:word.lower=' + word1.lower(),
            #'-1:word.istitle=%s' % word1.istitle(),
            #'-1:word.isupper=%s' % word1.isupper(),
            '-1:postag=' + postag1,
            #'-1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('BOS')
        
    if i < len(sent)-1:
        pass
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.extend([
            '+1:word.lower=' + word1.lower(),
            #'+1:word.istitle=%s' % word1.istitle(),
            #'+1:word.isupper=%s' % word1.isupper(),
            '+1:postag=' + postag1,
            #'+1:postag[:2]=' + postag1[:2],
        ])
    else:
        features.append('EOS')
                
    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]
def sent2labels(sent):
    return [label for token, label in sent]
def sent2tokens(sent):
    return [token for token, label in sent]

In [6]:
train_sents = readCorpus("E:/Project Sentiment analysis/CRF/test.txt")
x_train = [sent2features(s) for s in train_sents]
y_train = [sent2labels(s) for s in train_sents]


print("start append train set.")
trainer = pycrfsuite.Trainer(verbose=False)
for xseq, yseq in zip(x_train, y_train):
    trainer.append(xseq, yseq)
print("append train set done.")
trainer.set_params({
    'c1': 1.0,
    'c2': 1e-3,
    'max_iterations': 500,
    'feature.possible_transitions': True
    })

trainer.train("trained_model")
test_object = pycrfsuite.Tagger()
test_object.open("trained_model")


start append train set.
append train set done.
